# Expense Analysis (10 Months)

This notebook analyzes personal expense transactions from `expenses.csv`. It focuses on **expenses only** (`type = DEBIT`) and uses the `amount` column with inverted sign so spend is positive.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import seaborn as sns
from IPython.display import display

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)


ModuleNotFoundError: No module named 'pandas'

## Load + Clean

In [ ]:
from pathlib import Path

csv_path = Path("expenses.csv")

df = pd.read_csv(csv_path, encoding="utf-8-sig")

# Standardize column names (strip whitespace)
df.columns = [c.strip() for c in df.columns]

# Parse date
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Coerce amount to numeric
df["amount"] = pd.to_numeric(df["amount"], errors="coerce")

# Filter out months with insufficient coverage (need data through at least the 25th)
df["month"] = df["date"].dt.to_period("M")
coverage_months = (
    df.groupby("month")["date"].max().dt.day >= 25
).loc[lambda s: s].index
df = df[df["month"].isin(coverage_months)].copy()

# Keep only expenses
expenses = df[df["type"].str.upper() == "DEBIT"].copy()

# Filter out selected categories: internal transfers/payments, investments
expenses = expenses[expenses["category"].astype(str).str.strip().str.lower() != "transfer/pmt"].copy()
expenses = expenses[expenses["category"].astype(str).str.strip().str.lower() != "investment"].copy()
expenses = expenses[expenses["category"].astype(str).str.strip().str.lower() != "rental inc"].copy()

# Spend should be positive
expenses["spend"] = -expenses["amount"]

# Derive month and year from date
expenses["year"] = expenses["date"].dt.year
expenses["month_start"] = expenses["date"].dt.to_period("M").dt.to_timestamp()

# Basic cleanup
expenses["category"] = expenses["category"].fillna("(uncategorized)").str.strip().str.lower()

# Sanity checks
print("rows (all):", len(df))
print("rows (expenses only):", len(expenses))
print("date NaT:", expenses["date"].isna().sum())
print("amount NaN:", expenses["amount"].isna().sum())

expenses.head()

## Overview

In [ ]:
total_spend = expenses["spend"].sum()

summary = pd.DataFrame({
    "total_spend": [total_spend],
    "avg_monthly_spend": [expenses.groupby("month_start")["spend"].sum().mean()],
    "transaction_count": [len(expenses)],
})

display(summary.style.format({"total_spend": "${:,.0f}", "avg_monthly_spend": "${:,.0f}"}))


## Monthly Trend

In [ ]:
monthly = (
    expenses.groupby("month_start", as_index=False)["spend"].sum()
    .sort_values("month_start")
)

monthly["mom_change"] = monthly["spend"].diff()
display(monthly.style.format({"spend": "${:,.0f}", "mom_change": "${:,.0f}"}))


In [ ]:
plt.figure()
plt.plot(monthly["month_start"], monthly["spend"], marker="o")
plt.title("Monthly Spend")
plt.xticks(rotation=45)
plt.gca().yaxis.set_major_formatter(StrMethodFormatter("${x:,.0f}"))
plt.tight_layout()

## Category Breakdown

In [ ]:
category_total = (
    expenses.groupby("category", as_index=False)["spend"].sum()
    .sort_values("spend", ascending=False)
)

display(category_total.head(10).style.format({"spend": "${:,.0f}"}))


In [ ]:
plt.figure()
sns.barplot(data=category_total.head(10), x="spend", y="category")
plt.title("Top 10 Categories by Spend")
plt.xlabel("")
plt.ylabel("")
plt.gca().xaxis.set_major_formatter(StrMethodFormatter("${x:,.0f}"))
plt.tight_layout()

### Category Share by Month

In [ ]:
category_month = (
    expenses.groupby(["month_start", "category"], as_index=False)["spend"].sum()
)

pivot = category_month.pivot(index="month_start", columns="category", values="spend").fillna(0)

# Limit to top categories for readability
top_cats = category_total.head(6)["category"].tolist()

pivot_top = pivot[top_cats].copy()
# Use string month labels to avoid Period/Datetime freq issues in plotting
pivot_top.index = pivot_top.index.to_period("M").astype(str)

pivot_top.plot(kind="bar", stacked=True)
plt.title("Monthly Spend by Top Categories")
plt.xlabel("")
plt.ylabel("")
plt.xticks(rotation=45)
plt.gca().yaxis.set_major_formatter(StrMethodFormatter("${x:,.0f}"))
plt.tight_layout()

## Notable Findings

In [ ]:
# Top months
max_month = monthly.loc[monthly["spend"].idxmax()]
min_month = monthly.loc[monthly["spend"].idxmin()]

# Largest MoM increase
mom_inc = monthly.loc[monthly["mom_change"].idxmax()] if monthly["mom_change"].notna().any() else None

insights = {
    "top_month": f"{max_month['month_start'].date()} (${max_month['spend']:.2f})",
    "lowest_month": f"{min_month['month_start'].date()} (${min_month['spend']:.2f})",
}

if mom_inc is not None:
    insights["largest_mom_increase"] = f"{mom_inc['month_start'].date()} (+${mom_inc['mom_change']:.2f})"

insights